In [123]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

In [136]:
df = pd.read_csv('data.csv')
train_size = int(len(df) * 0.9)
train = df['Close'][:train_size]
test = df['Close'][train_size:]

In [137]:
# url='https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
# df = pd.read_csv(url,sep=",")
# train_size = int(len(df) * 0.90)
# train = df['Consumption'][:train_size]
# test = df['Consumption'][train_size:]

# The Auto-Regressive Model

In [161]:
from statsmodels.tsa.ar_model import AutoReg # AR or AutoReg

model = AutoReg(train, lags=2000)
model_fit = model.fit()

c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:248: FutureWarning:

The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 



In [162]:
# No. of lag variables
print(model_fit.ar_lags) # .k_ar or .ar_lags
# coef. of lag variables
print(model_fit.params)

#model_fit.summary()

[   1    2    3 ... 1998 1999 2000]
intercept      0.085654
Close.L1       0.948203
Close.L2       0.009120
Close.L3       0.041767
Close.L4      -0.002165
                 ...   
Close.L1996   -0.021341
Close.L1997   -0.022020
Close.L1998    0.023233
Close.L1999    0.006174
Close.L2000    0.003029
Length: 2001, dtype: float64


In [163]:
df['train'] = model_fit.predict(start=0, end=train_size-1, dynamic=False)
df['predictions'] = model_fit.predict(start=train_size, end=len(df)-1, dynamic=False)
display(df)

,Date,Close,train,predictions
0,1980-05-01,105.46,NaN,NaN
1,1980-05-02,105.58,NaN,NaN
2,1980-05-05,106.38,NaN,NaN
3,1980-05-06,106.25,NaN,NaN
4,1980-05-07,107.18,NaN,NaN
...,...,...,...,...
10639,2022-07-13,3801.78,NaN,3520.717127
10640,2022-07-14,3790.38,NaN,3497.083235
10641,2022-07-15,3863.16,NaN,3481.604476
10642,2022-07-18,3830.85,NaN,3472.480380


In [164]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(df['Close'][train_size:], df['predictions'][train_size:])
# can compare mse to naive forecast's
fig = px.line(df, y=["Close", "train", "predictions"], title=f"Auto-Regressive Forecast with MSE = {mse}")
fig.show()